In [13]:
import os
import train_model
from aws_helpers import create_repository, setup_elb, create_ecs_service, deregister_old_taskdefinitions
from keras_to_tfserving import convert_keras_to_tf_model

## Requirements
This code requires a couple of things:
- Installed requirements
- Installed Docker (https://docs.docker.com/install/linux/docker-ce/ubuntu/)
- Installed & configured AWS CLI (https://aws.amazon.com/cli/)
- AWS Elastic Load Balancer setup as an Application Load Balancer with a listener on 443 (and an SSL certificate)
- AWS Elastic Container Service cluster setup and configured

## Setting up a new Service

In [14]:
version = 1
service_path = 'services.dev.sellpy.net'
service_name = "stockholm-ai"
cluster = "microservices"
env = "dev"
load_balancer_name = "sellpy-services"

### Train model & Convert model

In [15]:
model_name = "stockholm_ai_mnist.m"  

if model_name not in os.listdir("."):
    train_model.output_model(model_name, epochs=1)

!python3 keras_to_tfserving.py stockholm_ai_mnist.m models/stockholm-ai

# TODO: Make this repeatable in notebook
#convert_keras_to_tf_model(model_name=model_name,
#                          model_path="models/stockholm-ai", # Note that the model path is referred to in server.conf
#                          sequential=True)

/home/maxber/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
['stockholm_ai_mnist.m', 'models/stockholm-ai']
2018-03-26 14:30:19.111202: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-03-26 14:30:19.111231: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-03-26 14:30:19.111255: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are availab

### Create a repository on ECR, build & push the image

In [16]:
# Create an image repository for storing docker files.
create_repository(service_name)

# Shellscript.
# could write it like: subprocess.call("<command>", shell=True)
!eval $( aws ecr get-login --no-include-email)
!docker build -t temp_image .
!docker tag temp_image 966836717103.dkr.ecr.eu-west-1.amazonaws.com/stockholm-ai:dev
!docker push 966836717103.dkr.ecr.eu-west-1.amazonaws.com/stockholm-ai:dev


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Login Succeeded
Sending build context to Docker daemon  53.45MB
Step 1/10 : FROM ubuntu:16.04
 ---> f975c5035748
Step 2/10 : RUN apt-get update && apt-get install -y         build-essential         curl         git         &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 1b8db2213203
Step 3/10 : ADD install.sh /tmp/install.sh
 ---> Using cache
 ---> 2f8661714cd5
Step 4/10 : RUN sh -e /tmp/install.sh
 ---> Using cache
 ---> 705ffe62372f
Step 5/10 : COPY requirements.txt /tmp/
 ---> Using cache
 ---> c5271e8a6b4e
Step 6/10 : RUN pip3 install -r /tmp/requirements.txt
 ---> Using cache
 ---> 3401bd12a327
Step 7/10 : RUN add-apt-repository ppa:ubuntu-toolchain-r/test -y
 ---> Using cache
 ---> 2bfd47cd6b86
Step 8/10 : RUN apt-get update -y && apt-get upgrade -y && apt-get dist-upgrade -y
 ---> Using cache
 ---> 853316a548c1
Step 9/10 : WORKDIR /app
 ---> Using cache
 ---> 9764e2987889
Ste

### Register a target & listener in the load balancer

In [6]:
target_group_arn = setup_elb(
    service=service_name,
    version=version,
    load_balancer_name=load_balancer_name,
    service_path=service_path
)

### Register a task definition and create an ECS service

In [8]:
# Create / Update a task definition.
!aws ecs register-task-definition --cli-input-json file://task_definition.json --region eu-west-1

create_ecs_service(
    cluster=cluster,
    service=service_name,
    task_definition_family=service_name,
    target_group_arn=target_group_arn,
    service_count=1
)

deregister_old_taskdefinitions(service_name)

{
    "taskDefinition": {
        "taskDefinitionArn": "arn:aws:ecs:eu-west-1:966836717103:task-definition/stockholm-ai:20",
        "containerDefinitions": [
            {
                "name": "stockholm-ai",
                "image": "966836717103.dkr.ecr.eu-west-1.amazonaws.com/stockholm-ai:dev",
                "cpu": 0,
                "memoryReservation": 300,
                "links": [
                    "serve_tensorflow_sthlm_ai:serve_tensorflow"
                ],
                "portMappings": [
                    {
                        "containerPort": 8080,
                        "hostPort": 1912,
                        "protocol": "tcp"
                    }
                ],
                "essential": true,
                "command": [
                    "python3",
                    "server.py"
                ],
                "environment": [
                    {
                        "name": "MNIST_SECRET_KEY",
                        "value": "sto

## Update existing services

In [19]:
#rebuild image
!eval $( aws ecr get-login --no-include-email)
!docker build -t temp_image .
!docker tag temp_image 966836717103.dkr.ecr.eu-west-1.amazonaws.com/stockholm-ai:dev
!docker push 966836717103.dkr.ecr.eu-west-1.amazonaws.com/stockholm-ai:dev

# Register a new Task def 
!aws ecs register-task-definition --cli-input-json file://task_definition.json --region eu-west-1
# Update the service with a new task def
!aws ecs update-service --service stockholm-ai --task-definition stockholm-ai --region eu-west-1  --cluster microservices
# Deregister tasks
deregister_old_taskdefinitions(service_name)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
Login Succeeded
Sending build context to Docker daemon  53.45MB
Step 1/10 : FROM ubuntu:16.04
 ---> f975c5035748
Step 2/10 : RUN apt-get update && apt-get install -y         build-essential         curl         git         &&     apt-get clean &&     rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 1b8db2213203
Step 3/10 : ADD install.sh /tmp/install.sh
 ---> Using cache
 ---> 2f8661714cd5
Step 4/10 : RUN sh -e /tmp/install.sh
 ---> Using cache
 ---> 705ffe62372f
Step 5/10 : COPY requirements.txt /tmp/
 ---> Using cache
 ---> c5271e8a6b4e
Step 6/10 : RUN pip3 install -r /tmp/requirements.txt
 ---> Using cache
 ---> 3401bd12a327
Step 7/10 : RUN add-apt-repository ppa:ubuntu-toolchain-r/test -y
 ---> Using cache
 ---> 2bfd47cd6b86
Step 8/10 : RUN apt-get update -y && apt-get upgrade -y && apt-get dist-upgrade -y
 ---> Using cache
 ---> 853316a548c1
Step 9/10 : WORKDIR /app
 ---> Using cache
 ---> 9764e2987889
Ste